### Configuração do ambiente de avaliação/inferência e hiperparâmetros de geração

Este bloco prepara, Liberum, as **constantes e parâmetros** usados na fase de **avaliação** (ou inferência controlada) do modelo ajustado, com foco em **reprodutibilidade** e **baixo consumo de VRAM** (≈8 GB).

**Entradas**
- N/A (somente define variáveis/constantes para uso posterior).

**Processo (passo a passo)**
1. **Imports e seed**  
   - Carrega bibliotecas de NLP/treinamento (Transformers, Datasets, PEFT, etc.).  
   - Define `set_seed(42)` para reprodutibilidade em geração e amostragem de subsets.
2. **Modelos e dados**  
   - `BASE_MODEL`: nome do modelo base usado como referência nas métricas.  
   - `ADAPTER_PATH`: caminho do **adapter LoRA** treinado (saída do SFT).  
   - `DATA_PATH`: caminho do dataset de teste (JSON com `title`, `content`).
3. **Idioma de referência**  
   - `LANG = "en"` (ajuste para `"pt"` se suas referências/respostas-alvo estiverem em português).
4. **Limites amigáveis para 8 GB de VRAM**  
   - `USE_4BIT = True`: habilita configuração pensada para quantização 4-bit (será usada na carga do modelo).  
   - `MAX_LEN = 512`: limita o comprimento do prompt/entrada para caber em GPUs menores.  
   - `VAL_SAMPLES` e `TEST_SAMPLES`: tamanhos de amostra para acelerar validação/teste.
5. **Parâmetros de geração** (`GEN_KWARGS`)  
   - `max_new_tokens=256`: controla o tamanho máximo da resposta gerada.  
   - `do_sample=False`: usa **decodificação gulosa** (determinística).  
   - `temperature=0.2`, `top_p=0.9`: definidos, mas **não têm efeito quando `do_sample=False`** (ficam inativos).

**Saídas**
- Variáveis globais configuradas (`BASE_MODEL`, `ADAPTER_PATH`, `DATA_PATH`, `LANG`, `USE_4BIT`, `MAX_LEN`, `VAL_SAMPLES`, `TEST_SAMPLES`, `GEN_KWARGS`) prontas para o próximo bloco (carregamento do modelo/adapter e execução das métricas).

**Observações**
- Se seu conjunto de referências estiver em **português**, altere `LANG` para `"pt"` para alinhar a avaliação.  
- Considere aumentar `MAX_LEN` para 1024 apenas se a GPU suportar sem **OOM**.  
- Para respostas mais criativas/variadas, mude para `do_sample=True` e então `temperature/top_p` passam a ter efeito.  
- Garanta que `ADAPTER_PATH` aponte para o diretório correto do LoRA salvo (ex.: `./models/out-sft/final`).


In [ ]:
import os, json, math, random
from typing import Dict, List

import numpy as np
import torch
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed)
from peft import PeftModel
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import evaluate

set_seed(42)


BASE_MODEL   = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"   
ADAPTER_PATH = "./models/out-sft/final"              
DATA_PATH    = "./data/tst.json"  

LANG = "en"   # "en" ou "pt"


USE_4BIT = True
MAX_LEN  = 512            
VAL_SAMPLES  = 512      
TEST_SAMPLES = 512

GEN_KWARGS = dict(
    max_new_tokens=256,
    temperature=0.2,
    top_p=0.9,
    do_sample=False
)


c:\Users\PC\Documents\Tech Challenge 3 (1)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Carregamento do modelo base + aplicação do adapter LoRA (quantização 4-bit) para inferência

Este bloco monta o **pipeline de inferência** com foco em **baixo uso de VRAM** (8 GB), habilitando quantização 4-bit via *bitsandbytes* e aplicando o **adapter LoRA** treinado sobre o modelo base.

**Entradas**
- `BASE_MODEL`: identificador do modelo base no Hugging Face Hub.
- `ADAPTER_PATH`: diretório do adapter LoRA salvo após o SFT.
- `USE_4BIT`: habilita/desabilita quantização 4-bit.

**Processo (passo a passo)**
1. **Configuração 4-bit (opcional)**  
   - Cria `BitsAndBytesConfig` com `nf4`, *double quant* e `compute_dtype=torch.float16`, reduzindo significativamente a VRAM.
2. **Tokenizer**  
   - `padding_side="left"` (recomendado para *causal LM* em geração batelada).  
   - Define `pad_token = eos_token` caso inexistente, evitando avisos/erros no *batching*.
3. **Dicas de performance**  
   - `torch.backends.cuda.matmul.allow_tf32 = True` habilita TF32 em GPUs Ampere/Hopper para *matmul* em FP32 (acelera operações compatíveis).  
   - `torch.set_grad_enabled(False)` desativa *autograd* (inferência pura).
4. **Carregamento do modelo base**  
   - `AutoModelForCausalLM.from_pretrained(...)` com `torch_dtype` ajustado ao hardware (FP16 com CUDA; FP32 na CPU), `quantization_config=bnb_config` e `device_map="auto"` (coloca o modelo automaticamente no(s) dispositivo(s) disponível(is)).
5. **Aplicação do adapter LoRA**  
   - `PeftModel.from_pretrained(base_model, ADAPTER_PATH)` carrega os pesos LoRA no topo do modelo base.  
   - `model.eval()` coloca o modelo em modo de avaliação (desativa *dropout*).
6. **Confirmação de dispositivo**  
   - Imprime o `device` do primeiro parâmetro do modelo para checagem rápida.

**Saídas**
- Objeto `model` pronto para geração (ex.: `model.generate(...)`) com quantização 4-bit (se habilitada) e LoRA aplicado.
- `tokenizer` configurado com *left padding*.

**Observações**
- A quantização 4-bit depende do pacote **`bitsandbytes`**; garanta que esteja instalado e compatível com sua GPU/driver.  
- TF32 acelera operações FP32; como a inferência aqui usa FP16, o ganho pode ser limitado, mas a flag é inofensiva.  
- Se ocorrer **OOM**, reduza `MAX_LEN`/`max_new_tokens` ou desative camadas não essenciais.  
- Para máxima compatibilidade, mantenha o `tokenizer` e o `model` do **mesmo checkpoint base** usado no treinamento do adapter.


In [2]:
# Quantização 4-bit (recomendado em 8GB)
bnb_config = None
if USE_4BIT:
    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True, padding_side="left")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Dicas de perf
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_grad_enabled(False)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

print("Device:", next(model.parameters()).device)


`torch_dtype` is deprecated! Use `dtype` instead!


Device: cuda:0


### Preparação do dataset de avaliação e criação de prompts em formato de chat

Este bloco, Liberum, carrega o dataset local a partir de JSON, realiza uma divisão **estratificada por seed** em `train/validation/test` e constrói um formato de **chat** (system/user → assistant) para avaliação/geração, produzindo pares `{prompt, reference}`.

**Entradas**
- `DATA_PATH`: caminho para um JSON com colunas `title` e `content`.
- `LANG`: idioma alvo dos prompts (`"en"` ou `"pt"`).
- `tokenizer`: já carregado previamente (usado para `apply_chat_template`).

**Processo (passo a passo)**
1. **Carga do dataset**  
   - `load_dataset("json", data_files=DATA_PATH, split="train")` lê o JSON como um único split.
2. **Split determinístico**  
   - `train_test_split(test_size=0.1, seed=42)` separa ~10% para **test**.  
   - Nova divisão no restante para criar **validation** (~10% do restante ≈ 0.1 total) e **train** (~80% total), via `train_test_split(test_size=0.1111, seed=42)`.  
   - Resultado final aproximado: **80% train / 10% validation / 10% test**.
3. **Template de chat**  
   - `build_chat_prompt(title, lang)` cria mensagens `system` e `user` diferentes conforme `LANG`:
     - EN: “Write a detailed description…”  
     - PT: “Escreva uma descrição detalhada…”
   - `tokenizer.apply_chat_template(..., tokenize=False, add_generation_prompt=True)` serializa as mensagens no formato esperado pelo modelo e adiciona o prefixo para a resposta do **assistant**.
4. **Mapeamento para avaliação**  
   - `format_example_chat` transforma cada exemplo em:
     - `prompt`: texto de entrada no formato de chat (com título e instruções).
     - `reference`: o `content` original, usado como referência para métricas/checagem de qualidade.
   - Aplica o `map` em cada split e monta `dataset_chat`.
5. **Inspeção**  
   - Visualiza o primeiro item de `validation` para checagem rápida do schema.

**Saídas**
- `dataset`: `DatasetDict` com `train`, `validation`, `test`.
- `dataset_chat`: `DatasetDict` com campos `prompt` e `reference` em cada split.
- Um exemplo impresso de `dataset_chat["validation"][0]` para verificação.

**Observações**
- A seed (`42`) garante **reprodutibilidade** do split.  
- `add_generation_prompt=True` é importante para modelos chat-based que esperam o marcador de início da fala do **assistant**.  
- Certifique-se de que os títulos (`title`) sejam informativos; prompts genéricos podem degradar a avaliação.  
- O `reference` serve como “ground truth” textual; métricas do tipo ROUGE/BLEU/BERTScore podem ser aplicadas em etapa posterior.


In [3]:
# Carrega dataset local (JSON com colunas: title, content)
raw = load_dataset("json", data_files=DATA_PATH, split="train")

# Split train/val/test
dataset = raw.train_test_split(test_size=0.1, seed=42)
tmp = dataset["train"].train_test_split(test_size=0.1111, seed=42)  # ~10% val, ~80% train
dataset = DatasetDict({
    "train": tmp["train"],
    "validation": tmp["test"],
    "test": dataset["test"]
})

def build_chat_prompt(title: str, lang: str = "en") -> str:
    if lang == "en":
        system = "You are a helpful assistant that writes detailed product descriptions from a given product title."
        user   = f"TITLE: {title}\nQuestion: Write a detailed description for this product."
    else:  # pt
        system = "Você é um assistente que escreve uma descrição detalhada do produto a partir do título fornecido."
        user   = f"TÍTULO: {title}\nPergunta: Escreva uma descrição detalhada para este produto."

    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
    ]
    # add_generation_prompt=True adiciona o prefixo para a resposta do assistant
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def format_example_chat(example, lang="en"):
    return {
        "prompt": build_chat_prompt(example["title"], lang=lang),
        "reference": example["content"]
    }

dataset_chat = DatasetDict({
    split: ds.map(lambda ex: format_example_chat(ex, lang=LANG))
    for split, ds in dataset.items()
})

# Visualiza um exemplo
dataset_chat["validation"][0]


Map: 100%|██████████| 97024/97024 [00:08<00:00, 11292.35 examples/s]


{'uid': 'B00CHH2SG8',
 'title': 'Crazy Bananas 6 Pieces Candy Scented Nail Polish Set, (Cherry, Blueberry, Grape, Strawberry, Banana, Apple)',
 'content': 'Crazy Bananas 6 piece candy scented nail polish. Kids fun nail polish. 0.24 fluid ounce each. Great gift.',
 'target_ind': [46466,
  52639,
  56909,
  102146,
  107626,
  110728,
  113886,
  113964,
  276873,
  304324,
  307464,
  344542,
  380682,
  381982,
  392963,
  503975,
  514450,
  558488,
  561824,
  577354,
  587446,
  679152,
  728168,
  1069311],
 'target_rel': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'prompt': '<|system|>\nYou are a helpful assistant that writes detailed product descriptions from a given product title.</s>\n<|user|>\nTITLE: Crazy Bananas 6 Pieces Candy Scented Nail Polish Set, (Cherry, Blueberry, Grape, Strawberry, Banana, Apple)\nQuestion: Write a detailed description for this

### Tokenização com máscara de rótulos (targets-only) e cálculo de Perplexidade

Este bloco, Liberum, prepara os exemplos no formato **[prompt + referência]** para avaliar o modelo medindo **Perplexidade** apenas sobre o **alvo (referência)** — o **prompt é ignorado** na perda via máscara de rótulos `-100`.

**Entradas**
- `dataset_chat["validation"]`: com campos `prompt` e `reference`.
- `tokenizer`, `model`, `MAX_LEN`, `tokenizer.pad_token_id`.

**Processo (passo a passo)**
1. **`tokenize_for_ppl_only_target(batch)`**
   - Tokeniza `prompt` (`p_ids`) **sem** adicionar *special tokens* (o chat template já contém).
   - Tokeniza a `reference` antecedida por um espaço (`" " + r`) → ajuda na junção de tokens.
   - Concatena `ids = p_ids + r_ids` e **trunca** para `MAX_LEN`.
   - Cria `attention_mask` = 1 para todos os tokens válidos.
   - Monta `labels` com **`-100` para os tokens do prompt** (ignorados na loss) e **`r_ids`** como alvos. Ajusta o comprimento para casar com `ids`.
   - Retorna `input_ids`, `attention_mask`, `labels`.
2. **Mapeamento do split de validação**
   - Aplica a função acima com `batched=True`; remove colunas originais, gerando `tok_val_masked`.
3. **Padding dinâmico custom (`pad_to_max`)**
   - Calcula o comprimento máximo do lote e **preenche**:
     - `input_ids` com `pad_id`.
     - `attention_mask` com `0` no padding.
     - `labels` com `-100` no padding (mantém máscara de perda coerente).
4. **`compute_perplexity_masked(tok_dataset, batch_size=4)`**
   - Cria `DataLoader` com `collate_fn` que chama `pad_to_max`.
   - Faz **inferência sem gradiente** (`torch.no_grad()`), coletando `out.loss` por batch.
   - Calcula `mean_loss` e `perplexity = exp(clamp(mean_loss, 20.0))` (o clamp evita overflow numérico).

**Saídas**
- Dicionário com métricas: `{"val_loss": <float>, "perplexity": <float>}`.
- `tok_val_masked`: dataset tokenizado e rotulado para avaliação.

**Observações**
- **Importante**: usar `-100` nos rótulos do prompt garante que a loss/perplexidade meça **apenas** a qualidade da geração da resposta (referência), não da instrução.
- O espaço antes da referência evita junção indesejada de tokens (wordpiece/BPE).
- Se ocorrer **OOM**, reduza `MAX_LEN` ou `batch_size`.
- O padding manual garante consistência entre `input_ids`, `attention_mask` e `labels` já mascarados.


In [ ]:
def tokenize_for_ppl_only_target(batch):
    prompts = batch["prompt"]
    refs    = batch["reference"]

    input_ids_list, attn_masks_list, labels_list = [], [], []
    for p, r in zip(prompts, refs):
        # tokenize sem adicionar specials (chat template já contém tokens de chat)
        p_ids = tokenizer(p, add_special_tokens=False).input_ids
        r_ids = tokenizer(" " + r, add_special_tokens=False).input_ids  # espaço anteposto ajuda
        # concat e trunca
        ids = (p_ids + r_ids)[:MAX_LEN]
        am  = [1] * len(ids)
        # labels: -100 no prompt; r_ids como alvo
        labels = ([-100] * min(len(p_ids), len(ids))) + r_ids
        labels = labels[:len(ids)]  # garante mesmo comprimento que ids

        input_ids_list.append(ids)
        attn_masks_list.append(am)
        labels_list.append(labels)

    return {
        "input_ids": input_ids_list,
        "attention_mask": attn_masks_list,
        "labels": labels_list
    }

tok_val_masked = dataset_chat["validation"].map(
    tokenize_for_ppl_only_target, batched=True,
    remove_columns=dataset_chat["validation"].column_names
)

# Padding dinâmico custom (como já temos labels prontos com -100)
def pad_to_max(batch, pad_id, label_pad=-100):
    maxlen = max(len(x) for x in batch["input_ids"])
    def pad(seq, fill, L): return seq + [fill] * (L - len(seq))
    input_ids = [pad(x, pad_id, maxlen) for x in batch["input_ids"]]
    attn_mask = [pad(x, 0,      maxlen) for x in batch["attention_mask"]]
    labels    = [pad(x, label_pad, maxlen) for x in batch["labels"]]
    return {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attn_mask),
        "labels": torch.tensor(labels)
    }

def compute_perplexity_masked(tok_dataset, batch_size=4):
    loader = DataLoader(
        tok_dataset, batch_size=batch_size, shuffle=False,
        collate_fn=lambda b: pad_to_max(
            {k: [d[k] for d in b] for k in b[0].keys()},
            pad_id=tokenizer.pad_token_id, label_pad=-100
        )
    )
    losses = []
    for batch in tqdm(loader, desc="Perplexidade (targets-only)"):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        with torch.no_grad():
            out = model(**batch)
        losses.append(out.loss.item())
    mean_loss = float(np.mean(losses))
    ppl = float(math.exp(min(mean_loss, 20.0)))
    return {"val_loss": mean_loss, "perplexity": ppl}


Map: 100%|██████████| 97014/97014 [00:22<00:00, 4251.66 examples/s]


### Geração em lote e avaliação automática com ROUGE, SacreBLEU, chrF e METEOR

Este bloco, Liberum, executa **inferência batelada** a partir dos `prompts` e calcula métricas de **sobreposição n-gram** e **caracteres** para avaliar a qualidade das saídas do modelo em relação às referências.

**Entradas**
- `dataset_chat[<split>]` com campos `prompt` e `reference`.
- `tokenizer`, `model`, `MAX_LEN`, `GEN_KWARGS`, `LANG`.

**Processo (passo a passo)**
1. **Carregamento de métricas**  
   - `rouge`, `sacrebleu`, `chrf` via `evaluate.load`.  
   - `meteor` é opcional (controlado por `HAS_METEOR` — pode falhar por dependências).
2. **Geração em lote (`generate_batch`)**  
   - Tokeniza `prompts` com **padding** e **truncation** em `MAX_LEN`.  
   - Usa `model.generate` com `GEN_KWARGS`, definindo `eos_token_id` e `pad_token_id`.  
   - **Pós-processamento**: aplica heurística simples para extrair apenas a resposta do **assistant**:
     - Divide pelo marcador da pergunta (`"Question:"` ou `"Pergunta:"` conforme `LANG`).  
     - Tenta cortar após `"Assistant:"` se existir no template.  
     - Faz `strip()` do texto final.
3. **Avaliação de um split (`evaluate_split`)**  
   - Itera sobre o conjunto em minibatches (16 exemplos), chamando `generate_batch`.  
   - Agrega hipóteses (`hyps`) e referências (`refs`).  
   - Calcula:
     - **ROUGE** (`rouge1`, `rouge2`, `rougeL`) com *stemming*.  
     - **SacreBLEU** (formato requer `[[ref]]` por hipótese).  
     - **chrF** (nível de caracteres).  
     - **METEOR** (se disponível).
   - Retorna dicionário com as pontuações agregadas.

**Saídas**
- Dicionário com métricas:  
  `{"rouge1", "rouge2", "rougeL", "sacrebleu", "chrf", "meteor"}`.

**Observações**
- A heurística de recorte pode variar conforme o **chat template** do modelo; ajuste as âncoras se o checkpoint usar marcadores diferentes.  
- Se `do_sample=False` em `GEN_KWARGS`, a geração é **determinística** (útil para comparabilidade entre runs).  
- Ajuste `MAX_LEN` caso ocorram truncamentos severos do prompt (pode afetar as métricas).  
- `METEOR` pode exigir dependências extras (Java/NLTK data); se indisponível, o campo retorna `NaN`.  
- Para avaliações extensas, considere normalizar caixa, remover tags, e aplicar **tokenização consistente** entre hipóteses e referências.


In [ ]:
metric_rouge = evaluate.load("rouge")
metric_bleu  = evaluate.load("sacrebleu")
metric_chrf  = evaluate.load("chrf")
try:
    metric_meteor = evaluate.load("meteor")
    HAS_METEOR = True
except Exception:
    HAS_METEOR = False

def generate_batch(prompts: List[str]) -> List[str]:
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, **GEN_KWARGS,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.pad_token_id)

    # TinyLlama-Chat: a resposta vem após o add_generation_prompt; extrair só a parte do assistant
    # Heurística simples: cortar pela última ocorrência do chat template do 'assistant' (ou 'Resposta:')
    generations = []
    for i, prompt in enumerate(prompts):
        full = tokenizer.decode(outputs[i], skip_special_tokens=True)
        # Se o prompt contém "Question:"/ "Pergunta:", pegue o que vem depois:
        split_key = "Question:" if LANG == "en" else "Pergunta:"
        text = full.split(split_key)[-1] if split_key in full else full
        # Corte em "Assistant:" se o template inserir essa âncora; caso contrário, use tudo
        if "Assistant:" in text:
            text = text.split("Assistant:")[-1]
        generations.append(text.strip())
    return generations

def evaluate_split(ds: Dataset, max_samples: int = 1000) -> Dict[str, float]:
    n = min(max_samples, len(ds))
    refs, hyps = [], []
    for i in tqdm(range(0, n, 16), desc=f"Avaliando ({n} amostras)"):
        batch = ds[i:i+16]
        hyps.extend(generate_batch(batch["prompt"]))
        refs.extend(batch["reference"])

    rouge = metric_rouge.compute(predictions=hyps, references=refs, use_stemmer=True)
    bleu  = metric_bleu.compute(predictions=hyps, references=[[r] for r in refs])
    chrf  = metric_chrf.compute(predictions=hyps, references=refs)
    meteor_score = (metric_meteor.compute(predictions=hyps, references=refs)["meteor"]
                    if HAS_METEOR else float("nan"))

    return {
        "rouge1": rouge["rouge1"], "rouge2": rouge["rouge2"], "rougeL": rouge["rougeL"],
        "sacrebleu": bleu["score"], "chrf": chrf["score"], "meteor": meteor_score
    }


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Avaliação final: Perplexidade (targets-only), métricas em validação/teste e salvamento de resultados

Este bloco, Liberum, executa a **avaliação completa** do modelo: calcula **Perplexidade** mascarando o prompt (somente sobre a referência), mede **ROUGE/SacreBLEU/chrF/METEOR** nos splits de **validação** e **teste**, agrega tudo em um dicionário e salva em `metrics.json`, registrando também metadados do experimento e o tempo total.

**Entradas**
- `tok_val_masked`: dataset de validação tokenizado com `labels=-100` no prompt.
- `dataset_chat["validation"]` e `dataset_chat["test"]` (campos `prompt`/`reference`).
- Hiperparâmetros e configs já definidos: `VAL_SAMPLES`, `TEST_SAMPLES`, `MAX_LEN`, `GEN_KWARGS`, `BASE_MODEL`, `ADAPTER_PATH`, `LANG`.

**Processo (passo a passo)**
1. **Cronometragem**  
   - Inicia `start = time.time()` para medir a duração total.
2. **Perplexidade (targets-only)**  
   - (Opcional) Subamostra via `select(...)`; aqui usa até `range(8000)` (todos os itens disponíveis da validação).  
   - Chama `compute_perplexity_masked(subset, batch_size=4)` e obtém `{"val_loss", "perplexity"}`.
3. **Métricas na validação**  
   - `val_metrics = evaluate_split(..., max_samples=VAL_SAMPLES)`: calcula **ROUGE-1/2/L**, **SacreBLEU**, **chrF** e **METEOR** (se disponível).
4. **Métricas no teste**  
   - `test_metrics = evaluate_split(..., max_samples=TEST_SAMPLES)` com as mesmas métricas.
5. **Agregação e metadados**  
   - Monta `all_metrics` combinando métricas de validação (incluindo PPL), de teste e um bloco `meta` com:
     - `base_model`, `adapter_path`, `gen_kwargs`, `max_len_ctx`, `val_samples`, `test_samples`, `seed`, `lang`, `ppl_targets_only=True`.
6. **Persistência**  
   - Salva `all_metrics` em **`metrics.json`** (UTF-8, identado).
7. **Relato final**  
   - Imprime o tempo decorrido (minutos) e retorna/mostra `all_metrics`.

**Saídas**
- Arquivo `metrics.json` no diretório atual com:
  - `validation`: ROUGE/SacreBLEU/chrF/METEOR + `val_loss` e `perplexity`.  
  - `test`: ROUGE/SacreBLEU/chrF/METEOR.  
  - `meta`: configurações usadas na execução.
- Impressões no console com progresso e duração total.

**Observações**
- **Subamostragem**: para rodar mais rápido, reduza `VAL_SAMPLES/TEST_SAMPLES` e/ou a seleção do subset de PPL.  
- **PPL targets-only**: mede a qualidade de modelagem **apenas da resposta**, não do prompt (graças aos `-100`).  
- **Reprodutibilidade**: garantida pela `seed`; evite alterar `GEN_KWARGS` se quiser comparabilidade entre execuções.  
- **Desempenho**: ajuste `batch_size`/`MAX_LEN` em GPUs com menos VRAM para evitar OOM.  
- **Interpretação**: ROUGE e chrF olham sobreposição; SacreBLEU é mais rígido; METEOR (se ativo) pondera sinônimos/stemming.


In [7]:
import time
start = time.time()

# Perplexidade (targets-only) — pode subamostrar se quiser mais rápido:
# Ex: subset = tok_val_masked.select(random.sample(range(len(tok_val_masked)), min(2000, len(tok_val_masked))))
subset = tok_val_masked.select(range(8000))  # usa tudo da validação
ppl_masked = compute_perplexity_masked(subset, batch_size=4)

print("\n📊 Avaliando conjunto de validação...")
val_metrics  = evaluate_split(dataset_chat["validation"], max_samples=VAL_SAMPLES)

print("\n📈 Avaliando conjunto de teste...")
test_metrics = evaluate_split(dataset_chat["test"],        max_samples=TEST_SAMPLES)

all_metrics = {
    "validation": {**val_metrics, **ppl_masked},
    "test": test_metrics,
    "meta": {
        "base_model": BASE_MODEL,
        "adapter_path": ADAPTER_PATH,
        "gen_kwargs": GEN_KWARGS,
        "max_len_ctx": MAX_LEN,
        "val_samples": VAL_SAMPLES,
        "test_samples": TEST_SAMPLES,
        "seed": 42,
        "lang": LANG,
        "ppl_targets_only": True
    }
}

with open("metrics.json", "w", encoding="utf-8") as f:
    json.dump(all_metrics, f, indent=2, ensure_ascii=False)

print(f"\n✅ Concluído em {(time.time()-start)/60:.2f} min.")
all_metrics


Perplexidade (targets-only): 100%|██████████| 2000/2000 [05:34<00:00,  5.98it/s]



📊 Avaliando conjunto de validação...


Avaliando (512 amostras): 100%|██████████| 32/32 [08:37<00:00, 16.16s/it]



📈 Avaliando conjunto de teste...


Avaliando (512 amostras): 100%|██████████| 32/32 [08:36<00:00, 16.13s/it]



✅ Concluído em 22.91 min.


{'validation': {'rouge1': np.float64(0.11451536666927262),
  'rouge2': np.float64(0.033575988489244556),
  'rougeL': np.float64(0.08437911788180151),
  'sacrebleu': 1.778428374749,
  'chrf': 14.480274127841566,
  'meteor': np.float64(0.08817447109161045),
  'val_loss': 2.6654058856368064,
  'perplexity': 14.373782474018604},
 'test': {'rouge1': np.float64(0.11288198600130454),
  'rouge2': np.float64(0.037328557479277644),
  'rougeL': np.float64(0.08489180838075167),
  'sacrebleu': 2.3524919534353073,
  'chrf': 13.636847210005303,
  'meteor': np.float64(0.08694649118272195)},
 'meta': {'base_model': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
  'adapter_path': './models/out-sft/final',
  'gen_kwargs': {'max_new_tokens': 256,
   'temperature': 0.2,
   'top_p': 0.9,
   'do_sample': False},
  'max_len_ctx': 512,
  'val_samples': 512,
  'test_samples': 512,
  'seed': 42,
  'lang': 'en',
  'ppl_targets_only': True}}

### Amostragem qualitativa: geração e exportação de exemplos comparativos

Este bloco, Liberum, gera **exemplos qualitativos** da performance do modelo — comparando lado a lado o **prompt**, a **resposta gerada** e a **referência humana** — permitindo inspeção manual e revisão externa.

**Entradas**
- `dataset_chat["validation"]`: contém colunas `prompt` e `reference`.
- Função `generate_batch`: já definida para gerar respostas a partir de uma lista de prompts.
- Parâmetros:
  - `k`: número de exemplos a gerar (padrão = 5).
  - `seed`: semente para amostragem reprodutível.

**Processo (passo a passo)**
1. **Seleção de amostra aleatória**  
   - Usa `random.sample` com `seed` fixo para escolher `k` índices da validação.  
   - Extrai `prompts` e `references` correspondentes.
2. **Geração das respostas**  
   - Itera sobre cada `prompt` (com barra de progresso `tqdm`).  
   - Usa `generate_batch([prompt])` para obter a resposta do modelo (`preds`).
3. **Construção dos registros**  
   - Cria lista de dicionários com:
     - `"id"` — índice sequencial.  
     - `"prompt"` — texto de entrada.  
     - `"gerado"` — resposta do modelo.  
     - `"referencia"` — descrição original.
   - Retorna a lista `rows`.
4. **Impressão formatada dos exemplos**  
   - Exibe cada exemplo separando com `=` para legibilidade.  
   - Mostra:
     - 🧩 número do exemplo  
     - 📥 *Prompt*  
     - 🤖 *Texto gerado*  
     - 🎯 *Referência original*
5. **Exportação para análise externa**  
   - Salva os exemplos em `qualitative_samples_val.jsonl`, com uma linha JSON por exemplo.
   - Reforça a presença também do arquivo de métricas `metrics.json` (do bloco anterior).

**Saídas**
- Arquivo `qualitative_samples_val.jsonl` com `prompt`, `gerado`, `referencia` (UTF-8, não escapado).  
- Impressão de `k` exemplos no console para análise rápida.  
- Retorno de `rows` como lista Python de exemplos.

**Observações**
- A avaliação qualitativa é crucial para interpretar **qualitativamente** as métricas automáticas (ROUGE/BLUE/chrF).  
- Útil para revisores humanos verificarem **fidelidade ao prompt** e **qualidade linguística**.  
- Pode ser adaptado para exportar para CSV ou HTML, facilitando revisão colaborativa.  
- Se o modelo usa templates de chat diferentes, ajustar a heurística de extração em `generate_batch` melhora a legibilidade das respostas.


In [8]:
def show_examples(ds: Dataset, k: int = 5, seed: int = 42):
    random.seed(seed)
    idxs = random.sample(range(len(ds)), k)
    samples = ds.select(idxs)
    prompts = samples["prompt"]
    references = samples["reference"]

    print(f"\nGerando {k} exemplos...\n")

    preds = []
    for prompt in tqdm(prompts, desc="Gerando exemplos"):
        preds.extend(generate_batch([prompt]))

    rows = []
    for i, (p, g, r) in enumerate(zip(prompts, preds, references)):
        rows.append({
            "id": i,
            "prompt": p,
            "gerado": g,
            "referencia": r
        })
    return rows

qual_val = show_examples(dataset_chat["validation"], k=5)

for row in qual_val:
    print("\n" + "="*100)
    print(f"🧩 Exemplo {row['id']+1}")
    print("\n📥 PROMPT:\n", row["prompt"])
    print("\n🤖 GERADO:\n", row["gerado"])
    print("\n🎯 REFERÊNCIA:\n", row["referencia"])

# Exporta para revisão externa
with open("qualitative_samples_val.jsonl", "w", encoding="utf-8") as f:
    for row in qual_val:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")
print("\nArquivos exportados: metrics.json, qualitative_samples_val.jsonl")



Gerando 5 exemplos...



Gerando exemplos: 100%|██████████| 5/5 [00:38<00:00,  7.73s/it]


🧩 Exemplo 1

📥 PROMPT:
 <|system|>
You are a helpful assistant that writes detailed product descriptions from a given product title.</s>
<|user|>
TITLE: Strawberry Shortcake Baby-girls Infant 2 Piece Polka Dot Legging Set, Yellow, 18 Months
Question: Write a detailed description for this product.</s>
<|assistant|>


🤖 GERADO:
 Write a detailed description for this product. 
<|assistant|>
Product: Strawberry Shortcake Baby-girls Infant 2 Piece Polka Dot Legging Set, Yellow, 18 Months (Item # 180000000000)Description:This set includes a pair of shorts with polka dot print and a matching top. The shorts are made of 100% cotton and the top is made of 100% cotton. The shorts have a drawstring waist and the top has a snap closure. The set is machine washable.

🎯 REFERÊNCIA:
 

🧩 Exemplo 2

📥 PROMPT:
 <|system|>
You are a helpful assistant that writes detailed product descriptions from a given product title.</s>
<|user|>
TITLE: Maxam&reg; 52oz Stainless Steel Oversized Mug
Question: Write a 

### Leitura, resumo e interpretação das métricas finais de desempenho

Este bloco, Liberum, carrega o arquivo `metrics.json` gerado anteriormente, exibe os principais valores numéricos de **validação** e **teste**, e apresenta um breve guia para interpretar corretamente as métricas.

**Entradas**
- Arquivo `metrics.json` criado no bloco de avaliação final.  
  Contém as chaves:
  - `"validation"` → métricas + perplexidade.
  - `"test"` → métricas automáticas de geração.
  - `"meta"` → informações do experimento (modelo, seed, idioma etc.).

**Processo (passo a passo)**
1. **Leitura do arquivo**  
   - Abre `metrics.json` em UTF-8 e carrega com `json.load(f)`.
2. **Filtragem e arredondamento**  
   - Seleciona apenas valores numéricos (`float`/`int`) em cada split.  
   - Arredonda para 4 casas decimais para legibilidade.  
   - Imprime separadamente:
     - `== Validation ==`  
     - `== Test ==`

**Saídas**
- Impressão no console dos valores principais arredondados.
- Contexto interpretativo para leitura rápida das métricas.

**Observações**
- **Alta perplexidade** sugere que o modelo ainda não se ajustou bem ao estilo das respostas esperadas.  
- **ROUGE/chrF altos e PPL baixo** indicam melhor consistência textual e adequação ao domínio.  
- Se `LANG` for alterado para `"pt"` e suas referências forem em português, as pontuações tendem a subir (o modelo avalia em idioma consistente).  
- Para análises formais, recomenda-se armazenar versões com `seed`, `config` e data de execução.


In [ ]:
with open("metrics.json", "r", encoding="utf-8") as f:
    m = json.load(f)

print("\n== Validation ==")
print({k: round(v, 4) for k, v in m["validation"].items() if isinstance(v, (int, float))})
print("\n== Test ==")
print({k: round(v, 4) for k, v in m["test"].items() if isinstance(v, (int, float))})



== Validation ==
{'rouge1': 0.1145, 'rouge2': 0.0336, 'rougeL': 0.0844, 'sacrebleu': 1.7784, 'chrf': 14.4803, 'meteor': 0.0882, 'val_loss': 2.6654, 'perplexity': 14.3738}

== Test ==
{'rouge1': 0.1129, 'rouge2': 0.0373, 'rougeL': 0.0849, 'sacrebleu': 2.3525, 'chrf': 13.6368, 'meteor': 0.0869}
